In [0]:
%pip install --upgrade streamlit databricks-langchain openai
dbutils.library.restartPython()


In [0]:
import streamlit as st
import os
from openai import OpenAI
from databricks_langchain import DatabricksVectorSearch

# -----------------------------
# Configuration
# -----------------------------
VECTOR_SEARCH_ENDPOINT = "pizza-rag-endpoint"
VECTOR_INDEX_NAME = "retail_analytics.pizza.pizza_semantic_index"
LLM_MODEL = "databricks-meta-llama-3-1-8b-instruct"

DATABRICKS_HOST = "https://dbc-741c7540-6155.cloud.databricks.com"

# -----------------------------
# Authentication (Databricks-native)
# -----------------------------
# Works automatically inside Databricks Apps / Notebooks
if "OPENAI_API_KEY" not in os.environ:
    try:
        os.environ["OPENAI_API_KEY"] = (
            dbutils.notebook.entry_point
            .getDbutils()
            .notebook()
            .getContext()
            .apiToken()
            .get()
        )
    except Exception:
        pass

# -----------------------------
# Initialize clients (cached)
# -----------------------------
@st.cache_resource
def init_clients():
    vs = DatabricksVectorSearch(
        index_name=VECTOR_INDEX_NAME,
        endpoint=VECTOR_SEARCH_ENDPOINT
    )

    client = OpenAI(
        base_url=f"{DATABRICKS_HOST}/serving-endpoints"
    )

    return vs, client

vs, client = init_clients()

# -----------------------------
# RAG function (your logic)
# -----------------------------
def query_pizza_rag(question: str, k: int = 3) -> str:
    docs = vs.similarity_search(question, k=k)

    if not docs:
        return "No relevant data found to answer the question."

    context = "\n".join(doc.page_content for doc in docs)

    messages = [
        {
            "role": "system",
            "content": (
                "You are a senior business analyst. "
                "Answer strictly using the provided context. "
                "If the answer cannot be derived, say so explicitly."
            )
        },
        {
            "role": "user",
            "content": f"""
Context:
{context}

Question:
{question}

Answer with reasoning and metrics if available.
"""
        }
    ]

    response = client.chat.completions.create(
        model=LLM_MODEL,
        messages=messages,
        temperature=0.1,
        max_tokens=512
    )

    return response.choices[0].message.content

# -----------------------------
# Streamlit UI
# -----------------------------
st.set_page_config(
    page_title="Pizza Sales RAG Analytics",
    layout="wide"
)

st.title("🍕 Pizza Sales Analytics – RAG Assistant")
st.caption("Powered by Databricks Vector Search + LLM")

with st.sidebar:
    st.header("Settings")
    k = st.slider("Top-K documents", min_value=1, max_value=10, value=3)
    st.markdown("---")
    st.markdown("**Example questions:**")
    st.markdown("""
    • Which pizza category has the highest sales?  
    • What ingredients drive revenue the most?  
    • Which pizzas sell best at night?  
    """)

question = st.text_input(
    "Ask a business question about pizza sales:",
    placeholder="Which pizza category has the highest total sales?"
)

if st.button("Ask"):
    if not question.strip():
        st.warning("Please enter a question.")
    else:
        with st.spinner("Analyzing sales data..."):
            answer = query_pizza_rag(question, k=k)

        st.subheader("Answer")
        st.write(answer)


In [0]:
%sh
streamlit run /databricks/python_shell/scripts/db_ipykernel_launcher.py